In [1]:
import warnings
from pylj import comp, md, sample, util, em
import numpy as np
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

# Molecular dynamics: Testing the ideal gas law

## Molecular dynamics

Molecular dynamics is a powerful method for exploring the structure of solids, liquids, and gases. The idea is simple: calculate the forces acting on the atoms in a system and anlayse their time-dependent behaviour and motion using Newtonian mechanics. When eneroguh information on the motion of the individual particles has been gathered, it is possible to deduce properties of the system. 

The aim of this exercise to use molecular dynamics simulation to investigate the nautre of an ideal gas. This involves the 2-dimensional simulation of an argon gas analogue using [pylj](pythoninchemistry.org/pylj). The function defined below is a typical molecular dynamics simulation where the number of particles, volume of the system, and system temperature are kept constant (this is referred to as the NVT ensemble). 

In [2]:
def md_simulation(number_of_particles, temperature, box_length, number_of_steps, sample_frequency):
    # Creates the visualisation environment
    %matplotlib notebook
    # Initialise the system
    system = md.initialise(number_of_particles, temperature, box_length, 'square')
    # This sets the sampling class
    sample_system = sample.Scattering(system)
    # Start at time 0
    system.time = 0
    # Begin the molecular dynamics loop
    for i in range(0, number_of_steps):
        # At each step, calculate the forces on each particle 
        # and get acceleration
        system.particles, system.distances, system.forces = comp.compute_forces(system.particles, 
                                                                                system.distances,
                                                                                system.forces, system.box_length)
        # Run the equations of motion integrator algorithm
        system = md.velocity_verlet(system)
        # Allow the system to interact with a heat bath
        system = comp.heat_bath(system, temperature)
        # Iterate the time
        system.time += system.timestep_length
        system.step += 1
        # At a given frequency sample the positions and plot the RDF
        if system.step % sample_frequency == 0:
            sample_system.update(system)  
    return system

This simulation could be run for 10 particles over 1000 steps, with a temperature of 1 using the following command. Note that the temperature and pressure are all in arbitrary units. 

In [3]:
system = md_simulation(50, 1, 8, 1000, 10)

AttributeError: 'tuple' object has no attribute 'temp_sum'

We are wanting to investigate the nature of the gas ideality, therefore we are considering the following relationship

$$ p = \frac{nRT}{V}, $$

where pressure, $p$, is the observable as it is given as an output plot by pylj and any of $n$, $T$, or $V$ can be varied to investigate this relationship. However, in pylj the system area (as it is 2D) is constrained to be $16^2$. This limited us to varying the number of particles, or the temperature. 

It is possible to return the average pressure during the entire simulation using the following command. 

In [ ]:
print("Average pressure = {:.2f}+/-{:.2f}".format(system.pressure.mean(), system.pressure.std()))

Therefore, hopefully it is evident how pylj could be used, along with a for loop to study the increase number of particles. 

In [ ]:
numb_parts = np.array([2, 5, 10, 20, 40, 60, 80, 100, 120])
pressure = np.zeros_like(numb_parts, dtype=np.float64)
pressure_std = np.zeros_like(numb_parts, dtype=np.float64)
for i in range(0, len(numb_parts)):
    system = md_simulation(numb_parts[i], 1, 5000, 10)
    pressure[i] = system.pressure.mean()
    pressure_std[i] = system.pressure.std()

It is then possible to plot the pressure against the number of particle in each simulation. The straight line of the ideal gas law can also be included, in the internal units of pylj the ideal gas constant has the following value.

$$ R = 1.42\times 10^{-2} $$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams['font.size']=16
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12

plt.errorbar(numb_parts, pressure, marker='o', yerr=pressure_std, 
             ls='', color='#34a5daff', markeredgecolor='black', markersize=12)
plt.plot(numb_parts, 0.0142 * 1 / 16 ** 2 * numb_parts, color='#34a5daff')
plt.show()

print(pressure.dtype)

The deviation from the ideal gas law is clear. The student could be asked to comment on the deviation. 

The van der Waals' equation includes corrections associated with the interactions between atoms, and has the form

$$ p = \frac{nRT}{V-nb} - a\frac{n^2}{V^2}, $$

where $a$ and $b$ are the corrections. We can used scipy.optimize to fit values of $a$ and $b$ for our system.

In [ ]:
from scipy.optimize import curve_fit

def vdW(x, a, b):
    return (1. / 256. * x) / (16. ** 2. - x * b) - (a * (x ** 2.) / (16. ** 2. ** 2.))

ab = curve_fit(vdW, numb_parts, pressure)
a = ab[0][0]
b = ab[0][1]

plt.errorbar(numb_parts, pressure, marker='o', yerr=pressure_std, 
             ls='', color='#34a5daff', markeredgecolor='black', markersize=12)
plt.plot(numb_parts, 0.0142 * 1 / 16 ** 2 * numb_parts, color='#34a5daff', label='Ideal Gaw Law', ls = '-')
plt.plot(np.arange(0, 120, 1), vdW(np.arange(0, 120, 1), a, b), color='#34a5daff', label='vdW Equation', ls = '--')

plt.xlabel('Number of particles')
plt.ylabel('Pressure')
plt.legend(frameon=False)
plt.show()
print("a = {:.3f} and b = {:.2f}".format(a, b))